# Feed RNN inputs with variable length

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, PackedSequence, pad_packed_sequence
from torch.autograd import Variable

In [2]:
# zero-padded input
x = Variable(torch.Tensor([
    [[1., 1.], [1., 1.], [1., 1.], [1., 1.], [1., 1.]],
    [[2., 2.], [2., 2.], [2., 2.], [0., 0.], [0., 0.]],
    [[3., 3.], [0., 0.], [0., 0.], [0., 0.], [0., 0.]]]))

# list of valid length of each batch
batch_sizes = [5, 3, 1]

In [3]:
# [batch_size, max_seq_len, input_size]
x

Variable containing:
(0 ,.,.) = 
  1  1
  1  1
  1  1
  1  1
  1  1

(1 ,.,.) = 
  2  2
  2  2
  2  2
  0  0
  0  0

(2 ,.,.) = 
  3  3
  0  0
  0  0
  0  0
  0  0
[torch.FloatTensor of size 3x5x2]

# pack_padded_sequence
## - Save values dynamically from zero-padded inputs
## - Frequently used as `import pack_padded sequence as pack`

* Args:
    * input (Variable)
    * lengths (list of int)
    * batch_first (bool)
    
    
* Return:
    * PackedSequence

## batch_first=True

In [4]:
packed_x = pack_padded_sequence(
    input=x,
    lengths=[5, 3, 1], # list of length of each batch
    batch_first=True # input shape: [batch_first, max_seq_len, input_size]
)
packed_x

PackedSequence(data=Variable containing:
    1     1
    2     2
    3     3
    1     1
    2     2
    1     1
    2     2
    1     1
    1     1
[torch.FloatTensor of size 9x2]
, batch_sizes=[3, 2, 2, 1, 1])

In [5]:
print(packed_x.data)
print(packed_x.batch_sizes)

Variable containing:
    1     1
    2     2
    3     3
    1     1
    2     2
    1     1
    2     2
    1     1
    1     1
[torch.FloatTensor of size 9x2]

[3, 2, 2, 1, 1]


## batch_first=False

In [6]:
x_T = x.transpose(0, 1)
x_T

Variable containing:
(0 ,.,.) = 
  1  1
  2  2
  3  3

(1 ,.,.) = 
  1  1
  2  2
  0  0

(2 ,.,.) = 
  1  1
  2  2
  0  0

(3 ,.,.) = 
  1  1
  0  0
  0  0

(4 ,.,.) = 
  1  1
  0  0
  0  0
[torch.FloatTensor of size 5x3x2]

In [7]:
packed_x_T = pack_padded_sequence(
    input=x_T,
    lengths=[5, 3, 1], # list of length of each batch
    batch_first=False # [max_seq_len, batch_first, input_size]
)
packed_x_T

PackedSequence(data=Variable containing:
    1     1
    2     2
    3     3
    1     1
    2     2
    1     1
    2     2
    1     1
    1     1
[torch.FloatTensor of size 9x2]
, batch_sizes=[3, 2, 2, 1, 1])

# PackedSequence

* internal

```
PackedSequence_ = namedtuple('PackedSequence', ['data', 'batch_sizes'])
class PackedSequence(PackedSequence_):
    pass
```
* Args:
    * data (Variable) <= **zero-padded** tensor
    * batch_sizes (list of int) <= **in decreasing order**

In [8]:
x

Variable containing:
(0 ,.,.) = 
  1  1
  1  1
  1  1
  1  1
  1  1

(1 ,.,.) = 
  2  2
  2  2
  2  2
  0  0
  0  0

(2 ,.,.) = 
  3  3
  0  0
  0  0
  0  0
  0  0
[torch.FloatTensor of size 3x5x2]

In [9]:
PackedSequence(x, batch_sizes)

PackedSequence(data=Variable containing:
(0 ,.,.) = 
  1  1
  1  1
  1  1
  1  1
  1  1

(1 ,.,.) = 
  2  2
  2  2
  2  2
  0  0
  0  0

(2 ,.,.) = 
  3  3
  0  0
  0  0
  0  0
  0  0
[torch.FloatTensor of size 3x5x2]
, batch_sizes=[5, 3, 1])

# RNN takes PackedSequence

In [10]:
linear = nn.Linear(2, 2)
rnn = nn.RNN(2, 2, batch_first=True)

In [11]:
packed_x

PackedSequence(data=Variable containing:
    1     1
    2     2
    3     3
    1     1
    2     2
    1     1
    2     2
    1     1
    1     1
[torch.FloatTensor of size 9x2]
, batch_sizes=[3, 2, 2, 1, 1])

In [12]:
packed_h = PackedSequence(linear(packed_x.data), packed_x.batch_sizes)
packed_h

PackedSequence(data=Variable containing:
 1.2840  0.2970
 2.3917  0.1153
 3.4993 -0.0664
 1.2840  0.2970
 2.3917  0.1153
 1.2840  0.2970
 2.3917  0.1153
 1.2840  0.2970
 1.2840  0.2970
[torch.FloatTensor of size 9x2]
, batch_sizes=[3, 2, 2, 1, 1])

In [13]:
packed_y, last_h = rnn(packed_h)
packed_y, last_h

(PackedSequence(data=Variable containing:
 0.9480 -0.8677
 0.9865 -0.9700
 0.9966 -0.9935
 0.9856 -0.6243
 0.9966 -0.8922
 0.9849 -0.7109
 0.9965 -0.9020
 0.9854 -0.6820
 0.9852 -0.6919
[torch.FloatTensor of size 9x2]
, batch_sizes=[3, 2, 2, 1, 1]),
 Variable containing:
 (0 ,.,.) = 
   0.9852 -0.6919
   0.9965 -0.9020
   0.9966 -0.9935
 [torch.FloatTensor of size 1x3x2])

# pad_packed_sequence
## - zero-pad inputs and make it Tensor again
## - Frequently used as `import pad_packed sequence as pad`

* Args:
    * sequence (PackedSequence)
    * batch_first (bool)


* Return:
    * output (tuple of Variable)
    * lengths (list of int)

In [14]:
packed_y

PackedSequence(data=Variable containing:
 0.9480 -0.8677
 0.9865 -0.9700
 0.9966 -0.9935
 0.9856 -0.6243
 0.9966 -0.8922
 0.9849 -0.7109
 0.9965 -0.9020
 0.9854 -0.6820
 0.9852 -0.6919
[torch.FloatTensor of size 9x2]
, batch_sizes=[3, 2, 2, 1, 1])

In [15]:
pad_packed_sequence(packed_y, batch_first=True)

(Variable containing:
 (0 ,.,.) = 
   0.9480 -0.8677
   0.9856 -0.6243
   0.9849 -0.7109
   0.9854 -0.6820
   0.9852 -0.6919
 
 (1 ,.,.) = 
   0.9865 -0.9700
   0.9966 -0.8922
   0.9965 -0.9020
   0.0000  0.0000
   0.0000  0.0000
 
 (2 ,.,.) = 
   0.9966 -0.9935
   0.0000  0.0000
   0.0000  0.0000
   0.0000  0.0000
   0.0000  0.0000
 [torch.FloatTensor of size 3x5x2], [5, 3, 1])